In [0]:
!pip install torch torchvision feather-format kornia pyarrow Pillow wandb --upgrade 
!pip install git+https://github.com/fastai/fastprogress  --upgrade
!pip install git+https://github.com/fastai/fastai_dev

In [0]:
!git clone https://github.com/sdoria/mish

Cloning into 'mish'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 46 (delta 21), reused 19 (delta 6), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [0]:

%cd mish
from rangerlars import *
from mish import *
from mxresnet import *
from ranger import *

/content/mish
Mish activation loaded...


In [0]:
from fastai2.basics import *
from fastai2.vision.all import *
from fastai2.callback.all import *
from fastai2.basics import defaults

In [0]:
src = untar_data(URLs.IMAGEWOOF)
items = get_image_files(src)
split_idx = GrandparentSplitter(valid_name='val')(items)
lbl_dict = dict(
  n02086240= 'Shih-Tzu',
  n02087394= 'Rhodesian ridgeback',
  n02088364= 'Beagle',
  n02089973= 'English foxhound',
  n02093754= 'Australian terrier',
  n02096294= 'Border terrier',
  n02099601= 'Golden retriever',
  n02105641= 'Old English sheepdog',
  n02111889= 'Samoyed',
  n02115641= 'Dingo'
)

In [0]:
n_gpus = num_distrib() or 1
nw = min(8, num_cpus()//n_gpus)

split_idx = GrandparentSplitter(valid_name='val')(items)
tfms = [[PILImage.create], [parent_label, lbl_dict.__getitem__, Categorize()]]

dsrc = DataSource(items, tfms, splits=split_idx)

batch_tfms = [Cuda(), IntToFloatTensor(), Normalize(*imagenet_stats)]

dbch = dsrc.databunch(after_item=[ToTensor(), Resize(128)], 
                      after_batch=batch_tfms, 
                      bs=64, num_workers=nw)

In [0]:
learn = Learner(dbch, mxresnet50(c_out=10, sa=True), opt_func=Adam, loss_func=LabelSmoothingCrossEntropy(),
                metrics=accuracy)

In [0]:
learn.fit_flat_cos(5, 4e-3, pct_start=0.72)

epoch,train_loss,valid_loss,accuracy,time
0,2.300824,3.326290,0.226000,02:22
1,2.070931,2.019429,0.262000,02:22
2,1.867251,1.840929,0.382000,02:22
3,1.699768,1.730356,0.428000,02:22
4,1.503892,1.481084,0.568000,02:22


# v1

In [0]:
from fastai.vision import *

In [0]:
path = untar_data(URLs.IMAGEWOOF)

In [0]:
data = (ImageList.from_folder(path)
        .split_by_folder(valid='val')
        .label_from_folder()
        .transform(size=128)
        .databunch(bs=64, num_workers=2)
        .normalize(imagenet_stats))


In [0]:
learn = Learner(data, mxresnet50(c_out=10, sa=True), opt_func=optim.Adam,
              loss_func=LabelSmoothingCrossEntropy(),
               metrics=[accuracy])

In [0]:
learn.fit_fc(5, 4e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.166673,2.258612,0.224000,02:55
1,1.955459,1.983938,0.348000,02:56
2,1.744949,1.809621,0.440000,02:56
3,1.613534,1.744130,0.422000,02:56
4,1.350349,1.372394,0.634000,02:56
